In [1]:
import pandas as pd
import requests
import os
from bs4 import BeautifulSoup
from time import sleep
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import warnings
import folium

# 경고 메시지 생략(무시)
warnings.filterwarnings('ignore')

# 그래프에서 한글처리 문제 해결
font_path = 'malgun.ttf'
font_name = font_manager.FontProperties(fname = font_path).get_name()
rc('font', family = font_name)

# 그래프에서 음수(minus) 기호 문제 해결
plt.rcParams['axes.unicode_minus'] = False

In [10]:
stock = pd.read_csv('csv/삼성전자_주식4.csv')
stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    50 non-null     object
 1   close   50 non-null     int64 
 2   open    50 non-null     int64 
 3   max     50 non-null     int64 
 4   min     50 non-null     int64 
 5   volumn  50 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 2.5+ KB


In [12]:
stock['date'][:10]

0    2024-01-10
1    2024-01-11
2    2024-01-12
3    2024-01-15
4    2024-01-16
5    2024-01-17
6    2024-01-18
7    2024-01-19
8    2024-01-22
9    2024-01-23
Name: date, dtype: object

In [13]:
#날짜 데이터가 되면 
#stock.loc['2024-01':'2024-03']
#stock.loc['2024']
#stock.loc['2024':'2026']
#식의 조회가 가능해진다.

In [14]:
#ISO date 포맷
#'YYYY-MM-DD'
#'YYYY-MM-DD'

In [17]:
stock2 = stock.copy()
stock2['new_date'] = pd.to_datetime(stock['date']) 
stock2['new_date'] # 하단 dtype의 차이를 확인할 것

0    2024-01-10
1    2024-01-11
2    2024-01-12
3    2024-01-15
4    2024-01-16
5    2024-01-17
6    2024-01-18
7    2024-01-19
8    2024-01-22
9    2024-01-23
10   2024-01-24
11   2024-01-25
12   2024-01-26
13   2024-01-29
14   2024-01-30
15   2024-01-31
16   2024-02-01
17   2024-02-02
18   2024-02-05
19   2024-02-06
20   2024-02-07
21   2024-02-08
22   2024-02-13
23   2024-02-14
24   2024-02-15
25   2024-02-16
26   2024-02-19
27   2024-02-20
28   2024-02-21
29   2024-02-22
30   2024-02-23
31   2024-02-26
32   2024-02-27
33   2024-02-28
34   2024-02-29
35   2024-03-04
36   2024-03-05
37   2024-03-06
38   2024-03-07
39   2024-03-08
40   2024-03-11
41   2024-03-12
42   2024-03-13
43   2024-03-14
44   2024-03-15
45   2024-03-18
46   2024-03-19
47   2024-03-20
48   2024-03-21
49   2024-03-22
Name: new_date, dtype: datetime64[ns]

In [18]:
stock2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      50 non-null     object        
 1   close     50 non-null     int64         
 2   open      50 non-null     int64         
 3   max       50 non-null     int64         
 4   min       50 non-null     int64         
 5   volumn    50 non-null     int64         
 6   new_date  50 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 2.9+ KB


In [19]:
stock2 = stock2.set_index('date')
stock2

,close,open,max,min,volumn,new_date
date,,,,,,
2024-01-10,73600,75000,75200,73200,20259529,2024-01-10
2024-01-11,73200,72900,73600,72700,57691266,2024-01-11
2024-01-12,73100,73000,74100,72800,13038939,2024-01-12
2024-01-15,73900,73200,74000,73200,13212339,2024-01-15
2024-01-16,72600,73500,73700,72500,14760415,2024-01-16
2024-01-17,71000,73100,73300,71000,22683660,2024-01-17
2024-01-18,71700,71600,72000,70700,17853397,2024-01-18
2024-01-19,74700,73500,74700,73000,23363427,2024-01-19
2024-01-22,75100,75900,76000,75000,19673375,2024-01-22


In [22]:
stock2.loc['2024-01-22':'2024-01-31']

,close,open,max,min,volumn,new_date
date,,,,,,
2024-01-22,75100,75900,76000,75000,19673375,2024-01-22
2024-01-23,75200,75700,75800,74300,14786224,2024-01-23
2024-01-24,74000,75200,75200,73500,12860661,2024-01-24
2024-01-25,74100,74200,74800,73700,11737747,2024-01-25
2024-01-26,73400,73700,74500,73300,11160062,2024-01-26
2024-01-29,74400,73800,75200,73500,13976521,2024-01-29
2024-01-30,74300,75000,75300,73700,12244418,2024-01-30
2024-01-31,72700,73400,74000,72500,15703560,2024-01-31


In [23]:
stock2.loc['2024-01'] #이건 에러 난다.

KeyError: '2024-01'

In [24]:
stock2 = stock2.set_index('new_date')
stock2.loc['2024-01':'2024-02']  # 이거는 된다. new_date가 date 이므로..

,close,open,max,min,volumn
new_date,,,,,
2024-01-10,73600,75000,75200,73200,20259529
2024-01-11,73200,72900,73600,72700,57691266
2024-01-12,73100,73000,74100,72800,13038939
2024-01-15,73900,73200,74000,73200,13212339
2024-01-16,72600,73500,73700,72500,14760415
2024-01-17,71000,73100,73300,71000,22683660
2024-01-18,71700,71600,72000,70700,17853397
2024-01-19,74700,73500,74700,73000,23363427
2024-01-22,75100,75900,76000,75000,19673375


In [31]:
stock2 = stock.copy()
new_date = pd.to_datetime(stock['date']) 
stock2.insert(1,'new_date',new_date)

In [34]:
stock2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      50 non-null     object        
 1   new_date  50 non-null     datetime64[ns]
 2   close     50 non-null     int64         
 3   open      50 non-null     int64         
 4   max       50 non-null     int64         
 5   min       50 non-null     int64         
 6   volumn    50 non-null     int64         
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 2.9+ KB


,date,new_date,close,open,max,min,volumn
0,2024-01-10,2024-01-10,73600,75000,75200,73200,20259529
1,2024-01-11,2024-01-11,73200,72900,73600,72700,57691266
2,2024-01-12,2024-01-12,73100,73000,74100,72800,13038939
3,2024-01-15,2024-01-15,73900,73200,74000,73200,13212339
4,2024-01-16,2024-01-16,72600,73500,73700,72500,14760415


In [35]:
stock2.head()

,date,new_date,close,open,max,min,volumn
0,2024-01-10,2024-01-10,73600,75000,75200,73200,20259529
1,2024-01-11,2024-01-11,73200,72900,73600,72700,57691266
2,2024-01-12,2024-01-12,73100,73000,74100,72800,13038939
3,2024-01-15,2024-01-15,73900,73200,74000,73200,13212339
4,2024-01-16,2024-01-16,72600,73500,73700,72500,14760415


In [37]:
print(stock2.loc[0,'date'],type(stock2.loc[0,'date']))
print(stock2.loc[0,'new_date'],type(stock2.loc[0,'new_date']))

2024-01-10 <class 'str'>
2024-01-10 00:00:00 <class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [41]:
stock3 = stock2.copy()
stock3.set_index('new_date',inplace = True)
stock3.drop('date', axis = 'columns',inplace=True)
stock3

,close,open,max,min,volumn
new_date,,,,,
2024-01-10,73600,75000,75200,73200,20259529
2024-01-11,73200,72900,73600,72700,57691266
2024-01-12,73100,73000,74100,72800,13038939
2024-01-15,73900,73200,74000,73200,13212339
2024-01-16,72600,73500,73700,72500,14760415
2024-01-17,71000,73100,73300,71000,22683660
2024-01-18,71700,71600,72000,70700,17853397
2024-01-19,74700,73500,74700,73000,23363427
2024-01-22,75100,75900,76000,75000,19673375


In [42]:
#205페이지 period data

dates = ['2019-01-01','2020-03-01','2021-08-01']
ts_dates = pd.to_datetime(dates)

In [43]:
ts_dates

DatetimeIndex(['2019-01-01', '2020-03-01', '2021-08-01'], dtype='datetime64[ns]', freq=None)

In [49]:
# AS(연초), A(연말), MS(월초), M(월말), D(일),H(시간), T(분), S(초), W(주)
pr_day = ts_dates.to_period(freq = 'D')
print(pr_day)

pr_month = ts_dates.to_period(freq = 'M')
print(pr_month)

pr_week = ts_dates.to_period(freq = 'W')
print(pr_week)

pr_week2 = ts_dates.to_period(freq = 'W-MON')
print(pr_week2)

pr_day3 = ts_dates.to_period(freq = '3D') #2일단위 주기
print(pr_day3)

PeriodIndex(['2019-01-01', '2020-03-01', '2021-08-01'], dtype='period[D]')
PeriodIndex(['2019-01', '2020-03', '2021-08'], dtype='period[M]')
PeriodIndex(['2018-12-31/2019-01-06', '2020-02-24/2020-03-01',
             '2021-07-26/2021-08-01'],
            dtype='period[W-SUN]')
PeriodIndex(['2019-01-01/2019-01-07', '2020-02-25/2020-03-02',
             '2021-07-27/2021-08-02'],
            dtype='period[W-MON]')
PeriodIndex(['2019-01-01', '2020-03-01', '2021-08-01'], dtype='period[3D]')


In [60]:
#pd.date_range 함수를 이용하면 기간을 매우 간단하게 코드를 작성할 수 있다.
#pd.date_range(start = '시작일', end = '종료일', periods = 수량,   --  end 와 수량은 둘중 하나 사용
#              freq = '주기', tz = 'Asia/Seoul')

#2024-01-01부터 2024-03-05 까지 일단위의 배열(목록)
ts_day1 = pd.date_range(start = '2024-01-01', end = '2024-01-10', freq = 'D')
print(ts_day1)

ts_day2 = pd.date_range(start = '2024-01-01', end = '2024-01-10', freq = 'D',
                        tz = 'Asia/Seoul')
print(ts_day2)

#2024-01-01부터 2024-02-29 까지 매주 토요일의 날짜
ts_week_sat = pd.date_range(start = '2024-01-01', end = '2024-02-29', freq = 'W-SAT')
print(ts_week_sat)

ts_week_sat2 = pd.date_range(start = '2024-01-01', periods = 10, freq = 'W-SAT')
print(ts_week_sat2)


DatetimeIndex(['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04',
               '2024-01-05', '2024-01-06', '2024-01-07', '2024-01-08',
               '2024-01-09', '2024-01-10'],
              dtype='datetime64[ns]', freq='D')
DatetimeIndex(['2024-01-01 00:00:00+09:00', '2024-01-02 00:00:00+09:00',
               '2024-01-03 00:00:00+09:00', '2024-01-04 00:00:00+09:00',
               '2024-01-05 00:00:00+09:00', '2024-01-06 00:00:00+09:00',
               '2024-01-07 00:00:00+09:00', '2024-01-08 00:00:00+09:00',
               '2024-01-09 00:00:00+09:00', '2024-01-10 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='D')
DatetimeIndex(['2024-01-06', '2024-01-13', '2024-01-20', '2024-01-27',
               '2024-02-03', '2024-02-10', '2024-02-17', '2024-02-24'],
              dtype='datetime64[ns]', freq='W-SAT')
DatetimeIndex(['2024-01-06', '2024-01-13', '2024-01-20', '2024-01-27',
               '2024-02-03', '2024-02-10', '2024-02-17', '2024-02-24',
   

In [61]:
#208 period 배열

In [64]:
pr_4hour = pd.period_range(start = '2024-01-01',end = '2024-01-03', freq = '4h')
pr_4hour

PeriodIndex(['2024-01-01 00:00', '2024-01-01 04:00', '2024-01-01 08:00',
             '2024-01-01 12:00', '2024-01-01 16:00', '2024-01-01 20:00',
             '2024-01-02 00:00', '2024-01-02 04:00', '2024-01-02 08:00',
             '2024-01-02 12:00', '2024-01-02 16:00', '2024-01-02 20:00',
             '2024-01-03 00:00'],
            dtype='period[4H]')

In [67]:
pr_2hour = pd.period_range(start = '2024-01-01', periods = 5,  freq = '2h')
pr_2hour

PeriodIndex(['2024-01-01 00:00', '2024-01-01 02:00', '2024-01-01 04:00',
             '2024-01-01 06:00', '2024-01-01 08:00'],
            dtype='period[2H]')

In [70]:
stock4 = stock2.copy()

In [72]:
#pandas에는 날짜형 데이터가 없어서,
import datetime #으로 불러와야 하고,
date1 = datetime.date(2024,3,1)
type(date1)

datetime.date

In [74]:
#얘를 이용해서,
print(date1.year)
print(date1.month) # 이렇게 불러올 수 있다.
print(date1.day)

2024
3
1


In [76]:
date2 = '2024-03-12' #이건 문자형인데.. 여기서 2024, 3, 12 를 뽑으려면
date2.split('-')
int(date2.split('-')[0]) # 이런식으로

2024

In [77]:
#또는
date2[0:4] #이렇게..

'2024'

In [80]:
import datetime
date1 = datetime.date(2024,3,1)
print(date1.year)
print(date1.month)

2024
3


In [83]:
date2 = '2024-03-12'
date3 = date2.split('-')
date3
print(int(date3[0]))
print(int(date3[1]))

2024
3


In [86]:
#또는
print(date2[0:4])
print(date2[5:7])

2024
03


In [88]:
stock4.head()

,date,new_date,close,open,max,min,volumn
0,2024-01-10,2024-01-10,73600,75000,75200,73200,20259529
1,2024-01-11,2024-01-11,73200,72900,73600,72700,57691266
2,2024-01-12,2024-01-12,73100,73000,74100,72800,13038939
3,2024-01-15,2024-01-15,73900,73200,74000,73200,13212339
4,2024-01-16,2024-01-16,72600,73500,73700,72500,14760415


In [94]:
print(type(stock4['new_date']))
stock4['year'] = stock4['new_date'].dt.year  # 뒤에 .str을 찍으면 str으로 인식하는 것처럼 .dt를 찍으면 dt속성 사용 가능
stock4['month'] = stock4['new_date'].dt.month
stock4['day'] = stock4['new_date'].dt.day
stock4.head()

<class 'pandas.core.series.Series'>


,date,new_date,close,open,max,min,volumn,year,month,day
0,2024-01-10,2024-01-10,73600,75000,75200,73200,20259529,2024,1,10
1,2024-01-11,2024-01-11,73200,72900,73600,72700,57691266,2024,1,11
2,2024-01-12,2024-01-12,73100,73000,74100,72800,13038939,2024,1,12
3,2024-01-15,2024-01-15,73900,73200,74000,73200,13212339,2024,1,15
4,2024-01-16,2024-01-16,72600,73500,73700,72500,14760415,2024,1,16


In [101]:
# 위 방법을 조금 어렵게 하는 방법
stock4 = stock2.copy()
stock4['date'] # 이건 문자data
date = stock4['date'].str.split('-')
date
# 이걸 year만 쭉 뽑고 싶으면
#date[0] # 이건 아니고..


0     [2024, 01, 10]
1     [2024, 01, 11]
2     [2024, 01, 12]
3     [2024, 01, 15]
4     [2024, 01, 16]
5     [2024, 01, 17]
6     [2024, 01, 18]
7     [2024, 01, 19]
8     [2024, 01, 22]
9     [2024, 01, 23]
10    [2024, 01, 24]
11    [2024, 01, 25]
12    [2024, 01, 26]
13    [2024, 01, 29]
14    [2024, 01, 30]
15    [2024, 01, 31]
16    [2024, 02, 01]
17    [2024, 02, 02]
18    [2024, 02, 05]
19    [2024, 02, 06]
20    [2024, 02, 07]
21    [2024, 02, 08]
22    [2024, 02, 13]
23    [2024, 02, 14]
24    [2024, 02, 15]
25    [2024, 02, 16]
26    [2024, 02, 19]
27    [2024, 02, 20]
28    [2024, 02, 21]
29    [2024, 02, 22]
30    [2024, 02, 23]
31    [2024, 02, 26]
32    [2024, 02, 27]
33    [2024, 02, 28]
34    [2024, 02, 29]
35    [2024, 03, 04]
36    [2024, 03, 05]
37    [2024, 03, 06]
38    [2024, 03, 07]
39    [2024, 03, 08]
40    [2024, 03, 11]
41    [2024, 03, 12]
42    [2024, 03, 13]
43    [2024, 03, 14]
44    [2024, 03, 15]
45    [2024, 03, 18]
46    [2024, 03, 19]
47    [2024, 

In [111]:
# 이걸 year만 쭉 뽑고 싶으면
date[0] # 이건 아니고..
stock4['year']  = date.str.get(0)  #-> 이런것도 된다.
stock4['year']  = stock4['year'].astype(int)
stock4['month'] = date.str.get(1)
stock4['month'] = stock4['month'].astype(int)
stock4['day']   = date.str.get(2)
stock4['day']   = stock4['day'].astype(int)
stock4.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      50 non-null     object        
 1   new_date  50 non-null     datetime64[ns]
 2   close     50 non-null     int64         
 3   open      50 non-null     int64         
 4   max       50 non-null     int64         
 5   min       50 non-null     int64         
 6   volumn    50 non-null     int64         
 7   year      50 non-null     int32         
 8   month     50 non-null     int32         
 9   day       50 non-null     int32         
dtypes: datetime64[ns](1), int32(3), int64(5), object(1)
memory usage: 3.4+ KB


In [112]:
stock4.set_index(['year','month','day'],inplace = True)
stock4

date   new_date  close   open    max    min    volumn
year month day                                                             
2024 1     10   2024-01-10 2024-01-10  73600  75000  75200  73200  20259529
           11   2024-01-11 2024-01-11  73200  72900  73600  72700  57691266
           12   2024-01-12 2024-01-12  73100  73000  74100  72800  13038939
           15   2024-01-15 2024-01-15  73900  73200  74000  73200  13212339
           16   2024-01-16 2024-01-16  72600  73500  73700  72500  14760415
           17   2024-01-17 2024-01-17  71000  73100  73300  71000  22683660
           18   2024-01-18 2024-01-18  71700  71600  72000  70700  17853397
           19   2024-01-19 2024-01-19  74700  73500  74700  73000  23363427
           22   2024-01-22 2024-01-22  75100  75900  76000  75000  19673375
           23   2024-01-23 2024-01-23  75200  75700  75800  74300  14786224
           24   2024-01-24 2024-01-24  74000  75200  75200  73500  12860661
           25   2024-01-25 2024-01-25  74100  74200  74800  73700  11737747
           26   2024-01-26 2024-01-26  73400  73700  74500  73300  11160062
           29   2024-01-29 2024-01-29  74400  73800  75200  73500  13976521
           30   2024-01-30 2024-01-30  74300  75000  75300  73700  12244418
           31   2024-01-31 2024-01-31  72700  73400  74000  72500  15703560
     2     1    2024-02-01 2024-02-01  73600  73000  74200  72900  19881033
           2    2024-02-02 2024-02-02  75200  74000  75200  73700  14955881
           5    2024-02-05 2024-02-05  74300  74200  74800  73500  19026021
           6    2024-02-06 2024-02-06  74400  74300  74700  73300  14559254
           7    2024-02-07 2024-02-07  75000  74600  75500  74300  16566445
           8    2024-02-08 2024-02-08  74100  75000  75200  73600  20810708
           13   2024-02-13 2024-02-13  75200  74800  75200  74400  21966745
           14   2024-02-14 2024-02-14  74000  73700  74300  73700  12434945
           15   2024-02-15 2024-02-15  73000  74200  74400  73000  14120600
           16   2024-02-16 2024-02-16  72800  73300  73400  72500  13444781
           19   2024-02-19 2024-02-19  73800  72800  73900  72800  12726404
           20   2024-02-20 2024-02-20  73300  73700  73700  72800  14681477
           21   2024-02-21 2024-02-21  73000  73400  73700  72900  11503495
           22   2024-02-22 2024-02-22  73100  73800  73900  72700  15208934
           23   2024-02-23 2024-02-23  72900  73600  74200  72900  16225166
           26   2024-02-26 2024-02-26  72800  72300  73200  72200  14669352
           27   2024-02-27 2024-02-27  72900  73100  73400  72700  13201981
           28   2024-02-28 2024-02-28  73200  72900  73900  72800  11795859
           29   2024-02-29 2024-02-29  73400  72600  73400  72000  21176403
     3     4    2024-03-04 2024-03-04  74900  74300  75000  74000  23210474
           5    2024-03-05 2024-03-05  73700  74600  74800  73700  19505125
           6    2024-03-06 2024-03-06  72900  73200  73500  72700  21547905
           7    2024-03-07 2024-03-07  72200  73100  73300  72200  14516963
           8    2024-03-08 2024-03-08  73300  72800  73400  72600  19271349
           11   2024-03-11 2024-03-11  72400  72900  73100  72300   9740504
           12   2024-03-12 2024-03-12  73300  72600  73500  72100  13011654
           13   2024-03-13 2024-03-13  74100  73700  74100  73500  15243134
           14   2024-03-14 2024-03-14  74300  74400  74500  73600  22545539
           15   2024-03-15 2024-03-15  72300  73400  73700  72300  22580555
           18   2024-03-18 2024-03-18  72800  72600  73000  72500  11520348
           19   2024-03-19 2024-03-19  72800  72300  73000  71700  15376066
           20   2024-03-20 2024-03-20  76900  73700  77200  73400  50106297
           21   2024-03-21 2024-03-21  79300  79200  79300  77700  44569799
           22   2024-03-22 2024-03-22  78900  79600  79900  77800  26614741

In [115]:
#stock4.loc['2024-01-15']  # 이렇게 조회가 안된다.
stock4.loc[(2024,1,15)] # 이렇게 tuple 형태만 가능

date                 2024-01-15
new_date    2024-01-15 00:00:00
close                     73900
open                      73200
max                       74000
min                       73200
volumn                 13212339
Name: (2024, 1, 15), dtype: object

In [116]:
stock4.loc[(2024,1,15):(2024,1,20)] # 이렇게도 가능

date   new_date  close   open    max    min    volumn
year month day                                                             
2024 1     15   2024-01-15 2024-01-15  73900  73200  74000  73200  13212339
           16   2024-01-16 2024-01-16  72600  73500  73700  72500  14760415
           17   2024-01-17 2024-01-17  71000  73100  73300  71000  22683660
           18   2024-01-18 2024-01-18  71700  71600  72000  70700  17853397
           19   2024-01-19 2024-01-19  74700  73500  74700  73000  23363427

In [117]:
# 지금까지 날짜 분리, 
# 지금부터 인덱싱하는 방법 p212

In [122]:
stock5 = stock2.copy()
stock5.set_index('new_date', inplace = True)
stock5.drop('date',axis = 'columns',inplace = True)
stock5.head()

,close,open,max,min,volumn
new_date,,,,,
2024-01-10,73600,75000,75200,73200,20259529
2024-01-11,73200,72900,73600,72700,57691266
2024-01-12,73100,73000,74100,72800,13038939
2024-01-15,73900,73200,74000,73200,13212339
2024-01-16,72600,73500,73700,72500,14760415


In [121]:
stock5.index


DatetimeIndex(['2024-01-10', '2024-01-11', '2024-01-12', '2024-01-15',
               '2024-01-16', '2024-01-17', '2024-01-18', '2024-01-19',
               '2024-01-22', '2024-01-23', '2024-01-24', '2024-01-25',
               '2024-01-26', '2024-01-29', '2024-01-30', '2024-01-31',
               '2024-02-01', '2024-02-02', '2024-02-05', '2024-02-06',
               '2024-02-07', '2024-02-08', '2024-02-13', '2024-02-14',
               '2024-02-15', '2024-02-16', '2024-02-19', '2024-02-20',
               '2024-02-21', '2024-02-22', '2024-02-23', '2024-02-26',
               '2024-02-27', '2024-02-28', '2024-02-29', '2024-03-04',
               '2024-03-05', '2024-03-06', '2024-03-07', '2024-03-08',
               '2024-03-11', '2024-03-12', '2024-03-13', '2024-03-14',
               '2024-03-15', '2024-03-18', '2024-03-19', '2024-03-20',
               '2024-03-21', '2024-03-22'],
              dtype='datetime64[ns]', name='new_date', freq=None)

In [126]:
stock5.head()

,close,open,max,min,volumn
new_date,,,,,
2024-01-10,73600,75000,75200,73200,20259529
2024-01-11,73200,72900,73600,72700,57691266
2024-01-12,73100,73000,74100,72800,13038939
2024-01-15,73900,73200,74000,73200,13212339
2024-01-16,72600,73500,73700,72500,14760415


In [136]:
#1. stock5 에서 2024.1.10 ~ 2024.1.25까지 주식 data를 조회
#2. 이게 된 상태에서 
#2. 2024.1.10~2024.1.25의 주식 데이터 중 시작(open), 마감(close) 컬럼만 조회
#3. 최근 5일 주식 데이터 중 시작(open), 마감(close) 컬럼만 조회

stock5.loc['2024-01-10':'2024-01-25']
stock5.loc['2024-01-10':'2024-01-25',['open','close']]
stock5.iloc[-5:,[1,0]]

,open,close
new_date,,
2024-03-18,72600,72800
2024-03-19,72300,72800
2024-03-20,73700,76900
2024-03-21,79200,79300
2024-03-22,79600,78900
